In [ ]:
import xarray as xr
import os

OBS_DATA_PATH = "../data/raw/obs/"
GRID_CLIM_PATH = "../data/thresholds/grid"
POINT_CLIM_PATH = "../data/thresholds/point"

In [ ]:
obs_file_list = sorted([f for f in os.listdir(OBS_DATA_PATH) if f.endswith(".nc")])

In [ ]:
obs_metadata = []
for obs_file_name in obs_file_list:
    obs = xr.open_dataarray(f"{OBS_DATA_PATH}{obs_file_name}")
    station = obs.attrs["station"]
    latitude = obs.attrs["lat"]
    longitude = obs.attrs["lon"]
    obs_meta_dict = {"station": station, "latitude": latitude, "longitude": longitude}
    obs_metadata.append(obs_meta_dict)

In [ ]:
file_names = [
    "clim_0.99.nc",
    "clim_0.999.nc",
]
for file in file_names:
    grid_clim = xr.open_dataarray(f"{GRID_CLIM_PATH}/{file}")
    point_quant_list = []
    for station in obs_metadata:
        point_quant = grid_clim.sel(
            latitude=station["latitude"],
            longitude=360 + station["longitude"],
            method="nearest",
        )
        point_quant = point_quant.expand_dims("station")
        point_quant = point_quant.assign_coords({"station": [station["station"]]})
        point_quant_list.append(point_quant)
    point_quants = xr.concat(point_quant_list, dim="station")
    point_quants.to_netcdf(f"{POINT_CLIM_PATH}/{file}")